In [ ]:
import irradiance_beds
import pandas as pd
import xarray as xr
import numpy as np

import pvdeg

In [ ]:
pvdeg.geospatial.start_dask()

coords = [
    (37.01, -108.9),
    (37.01, -108.74)
]

# # this is to fix the missing weather data
weather, meta, failed = pvdeg.weather.weather_distributed(
    coords = coords,
    database="PSM3",
    email="tobin.ford@nrel.gov"
)

Force load a sample output so we can generate an output template.

In [ ]:
conf = "01"
fp = f"/projects/inspire/PySAM-MAPS/CO-sample/{conf}/1-16th-res-{conf}.nc"

loaded = xr.open_dataset(fp)
loaded_sub = loaded.isel(latitude=slice(0,1), longitude=range(1,3))

template = irradiance_beds.output_template(loaded_sub)

In [ ]:
template.compute()

In [ ]:
for conf in ["01", "02", "03"]:

    conf_template = template.copy()
    
    for i in range(1,3):

        single_location_irrradiance_da = loaded.isel(latitude=0, longitude=i).ground_irradiance
        latitude, longitude = coords[i - 1]

        conf_beds = irradiance_beds.testbeds_irradiance(
            irradiance_df=single_location_irrradiance_da.to_pandas(),
            conf=conf
        )

        # save conf_beds results
        for var_name, da in conf_beds_ds.data_vars.items():
            conf_template[var_name].loc[dict(latitude=[latitude], longitude=[longitude])] = da

        # save irradiance data with distance axis
        conf_template['ground_irradiance'].loc[dict(latitude=[latitude], longitude=[longitude])] = single_location_irrradiance_da

        # save weather data|
        weather_da = weather.isel(gid=i - 1).drop_vars(["Year", "Month", "Day", "Hour", "Minute", "gid"])
        for var_name, da in weather_da.data_vars.items():
            conf_template[var_name].loc[dict(latitude=[latitude], longitude=[longitude])] = da

    # conf_template.to_netcdf(f"conf_{conf}-agrivoltaics-irradiance-weather.h5", engine="h5netcdf")

In [ ]:
#weather["time"] = res.index # overwrite mismatched time index, both are localized tmy data

for conf in ["01", "02", "03"]:
    for i in range(1,3):

        fp = f"/projects/inspire/PySAM-MAPS/CO-sample/{conf}/1-16th-res-{conf}.nc"

        single_location_irrradiance_da = loaded.isel(latitude=0, longitude=i).ground_irradiance
        latitude = single_location_irrradiance_da.latitude.values
        longitude = single_location_irrradiance_da.longitude.values

        single_location_tmy_df = weather.isel(gid=0).drop_vars(["Year", "Month", "Day", "Hour", "Minute", "gid"]).to_dataframe()

        conf_beds = irradiance_beds.testbeds_irradiance(
            irradiance_df=single_location_irrradiance_da.to_pandas(),
            conf=conf
        )

        
        
        

        #res = pd.concat([single_location_irrradiance_df, conf_beds, single_location_tmy_df], axis=1)

        #res.to_csv(f"beds-irradiance-{latitude}-{longitude}-conf-{conf}.csv")

In [ ]:
xr.Dataset.from_dataframe(conf_beds).expand_dims(["latitude", "longitude"])

In [ ]:
conf_beds = irradiance_beds.testbeds_irradiance(
    irradiance_df=single_location_irrradiance_da.to_pandas(),
    conf=conf
)


In [ ]:
conf = "01"
fp = f"/projects/inspire/PySAM-MAPS/CO-sample/{conf}/1-16th-res-{conf}.nc"

loaded = xr.open_dataset(fp)

irradiance_beds.output_template(loaded)

In [ ]:
lats = (37.01,)
lons = (-108.9, -108.74)

single_location = xr.Dataset.from_dataframe(single_location_tmy_df).expand_dims({"latitude":lats, "longitude":lons})

single_location["ground_irradiance"] = single_location_irrradiance_da

single_location

In [ ]:
xr.Dataset(
    
    
)

In [ ]:
weather.isel(gid=0).drop_vars(["Year", "Month", "Day", "Hour", "Minute", "gid"])

In [ ]:
fp = f"/projects/inspire/PySAM-MAPS/CO-sample/{conf}/1-16th-res-{conf}.nc"


loaded = xr.open_dataset(fp)

In [ ]:
loaded

In [ ]:
### CONVERTING TO H5 from csvs
# this is a bad step, we do not want to do this on a large scale but CSV's existed already ehhh

import os
import glob
import xarray as xr

files = glob.glob(os.path.join(os.getcwd(), "beds-*.csv"))

In [ ]:
files

In [ ]:
from collections import defaultdict
# Group by configuration type (conf-01, conf-02, conf-03)
config_groups = defaultdict(list)

for file in files:
    # Extract configuration type from the filename
    parts = file.split("-")
    config_type = parts[-1].split(".")[0]  # Extract conf-01, conf-02, conf-03
    config_groups[config_type].append(file)

    
# Display the grouped results
for config, paths in sorted(config_groups.items()):
    print(f"{config} group has {len(paths)} locations:")
    for path in paths:
        print(f"  - {path}")
    print()

In [ ]:
config_groups

In [ ]:
for conf in ['01','02','03']:
    path_a = config_groups[conf][0]
    path_a = config_groups[conf][1]

    single = load_csv(path_a)

    # only contains first location for now
    builder = xr.Dataset.from_dataframe(single).expand_dims({"latitude":lats, "longitude":lons})
    
    second_location = xr.Dataset.from_dataframe(load_csv(path_b))
    
    builder = builder.assign({
        var: builder[var].where(~((builder.latitude == lats[0]) & (builder.longitude == lons[1])), second_location[var])
        for var in second_location.data_vars
    })

    builder.to_netcdf(f"{conf}-irradiance.h5", engine="h5netcdf")

    

In [ ]:
xr.open_dataset("03-irradiance.h5").edgetoedge.plot()

In [ ]:
files = groups['--108.73999786376953']
files = groups['--108.9000015258789']

# handle configs in order
files.sort()

path_a = files[0]
path_b = files[1]

def load_csv(path) -> xr.Dataset:

    single = pd.read_csv(path, index_col=0)
    single.index = pd.date_range("2001-01-01 00:30:00", periods=8760, freq="1h")
    single.index.name = "time"

    return single

single = load_csv(path_a)
single

In [ ]:
lats = [
37.0099983215332
]
    
lons = [
-108.73999786376953,
-108.9000015258789
]

# only contains first location for now
builder = xr.Dataset.from_dataframe(single).expand_dims({"latitude":lats, "longitude":lons})

second_location = xr.Dataset.from_dataframe(load_csv(path_b))

builder = builder.assign({
    var: builder[var].where(~((builder.latitude == lats[0]) & (builder.longitude == lons[1])), second_location[var])
    for var in second_location.data_vars
})

In [ ]:
builder.isel(latitude=0, longitude=0)